# **1-1. 통합 테스트**

## **1.환경설정**

### (1) 경로 설정

로컬 경로 설정

In [7]:
path = './'

### (2) 라이브러리

In [8]:
#필요한 라이브러리 설치 및 불러우기
import os
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import openai
from openai import OpenAI
import json
import torch
import sys
path = './'
sys.path.append(path)

from transformers import AutoTokenizer, AutoModelForSequenceClassification


# 더 필요한 라이브러리 추가 -------------
import importlib
import modules_AI_06_19 as md
importlib.reload(md)

<module 'modules_AI_06_19' from 'c:\\Users\\User\\Mini7_team_fastapi\\ktaivle_mini7_fastapi\\modules_AI_06_19.py'>

## **2. 단위 테스트**

* 세부사항 : 아래 단계별로 데이터가 순차적으로 처리되도록 단위 테스트를 진행합니다.

### (1) open ai key 등록

In [6]:

def load_api_keys(filename):
    with open(filename, 'r') as file:
        keys = json.load(file)
    return keys
openapi=load_api_keys("keys.json")

openai.api_key = openapi['openapi']
service = openapi['service']
c_id, c_key = openapi['c_id'], openapi['c_key']

os.environ['OPENAI_API_KEY'] = openai.api_key
os.environ['SERVICE_KEY'] = service
os.environ['CLIENT_ID'] = c_id
os.environ['CLIENT_KEY'] = c_key

### (2) audio to text

In [9]:
audio_path = path + 'audio/'
filename = 'merged_audio4.wav'

api_key = os.environ.get('OPENAI_API_KEY')
A2T = md.AudioTextProcessor(api_key)

result = A2T.audio_to_text(audio_path, filename)


API Response: 위중한 상태인 것 같은데요. 지금 호흡이 곤란하세요. 지금 가면 응급처치 받을 수 있나요? 통증이 심하시거든요. 오른쪽 팔에 쏘이셨어요. 아버지가 별촌하다가 말벌에 쏘이셨어요.



### (3) text summary

In [10]:
role = '''
      당신은 119에서 긴급전화를 받는 어시스턴트입니다.
      현재 입력된 텍스트는 급박하고 긴장되는 상황의 발언자가 하는 얘기이므로,
      응급 상황임을 감안하여 발언자의 텍스트를 요약해주세요

      KTAS 응급환자 중증도 분류기준에 대해 알아보고 위 기준에 따른 위험도 판단을 해줘
      위험도 판단은 소생, 긴급, 응급, 준응급, 비응급 5단계로 나누어주세요

      응답은 다음의 형식을 지켜주세요. JSON 형식으로 반환할 것
      {"summary": "텍스트 요약",
        "keyword": "핵심 키워드(3가지)",
        "danger": "위험도 단계 표시"}
    '''

In [11]:
summary = A2T.text_summary(role, result)
summary

TypeError: AudioTextProcessor.text_summary() takes 2 positional arguments but 3 were given

### (4) 응급실 등급분류

In [79]:
# 모델, 토크나이저 로드
save_directory = path + "fine_tuned_bert"
model = AutoModelForSequenceClassification.from_pretrained(save_directory)
tokenizer = AutoTokenizer.from_pretrained(save_directory)

In [80]:

Bert = md.ModelInstance(loadpath = path + 'fine_tuned_bert')

Bert.test(summary)



예측된 클래스: 1
예측된 클래스 이름: 2등급
클래스별 확률: tensor([[0.3380, 0.4589, 0.0468, 0.0812, 0.0751]])


### (5) 응급실추천

In [91]:
emergency = pd.read_csv(path + 'emergency_data.csv')

service = openapi['service']

c_id, c_key = openapi['c_id'], openapi['c_key']

In [146]:
audio_path = path + 'audio/'
filename = 'merged_audio4.wav'

point = pd.read_csv(path + 'audio_location.csv')


# 오디오 파일명에 해당하는 행을 찾기
matching_row = point[point['filename'] == filename]
matching_row
# 동적으로 위도와 경도 정보 할당
latitude = matching_row.iloc[0]['위도']
longitude = matching_row.iloc[0]['경도']


public_data_service = md.PublicData(service=service)

Recog = md.GetDistance(csv=path+'emergen_df.csv', c_id=c_id, c_key=c_key, public_data_service=public_data_service)
hospitals = Recog.recommend_hospital(latitude, longitude)

# for hospital in hospitals:
#     duty_code = hospital[3]

#     bed_count = public_data_service.fetch_emrg_bed_info(duty_code=duty_code)

#     if bed_count > 0:
#         print(f"Hospital with available beds: {hospital}")


KeyError: 'route'

## **3. 통합 테스트**

* 세부사항
    * [2. 단위 테스트] 의 내용을 순차적으로 실행하며 결과를 확인합니다.

In [12]:
# 1. 준비 작업 ---------------------------------------------
def load_api_keys(filename):
    with open(filename, 'r') as file:
        keys = json.load(file)
    return keys
openapi=load_api_keys("keys.json")

openai.api_key = openapi['openapi']
service = openapi['service']
c_id, c_key = openapi['c_id'], openapi['c_key']

os.environ['OPENAI_API_KEY'] = openai.api_key
os.environ['SERVICE_KEY'] = service
os.environ['CLIENT_ID'] = c_id
os.environ['CLIENT_KEY'] = c_key


emergency = pd.read_csv(path + 'emergen_df.csv')

point = pd.read_csv(path + 'audio_location.csv')

# 모델, 토크나이저 로드
save_directory = path + "fine_tuned_bert"
model = AutoModelForSequenceClassification.from_pretrained(save_directory)
tokenizer = AutoTokenizer.from_pretrained(save_directory)


# 2. 데이터 처리(pipeline) ---------------------------------

# input
audio_path = path + 'audio/'
filename = 'merged_audio1.wav'


# 처리
A2T = md.AudioTextProcessor(api_key=openai.api_key)
result = A2T.audio_to_text(audio_path, filename)

summary = A2T.text_summary(result)

Bert = md.ModelInstance(loadpath = path + 'fine_tuned_bert')
#model_result = Bert.test(summary)

# 응급실 추천

# 오디오 파일명에 해당하는 행을 찾기
matching_row = point[point['filename'] == filename]
matching_row
# 동적으로 위도와 경도 정보 할당
latitude = matching_row.iloc[0]['위도']
longitude = matching_row.iloc[0]['경도']


# 국립 중앙으료원 API / 네이버지도 API 호출, 거리에 의한 병원추천 함수 실행
public_data_service = md.PublicData(service=service)
Recog = md.GetDistance(csv=path+'emergen_df.csv', c_id=c_id, c_key=c_key, public_data_service=public_data_service)
hospitals = Recog.recommend_hospital(latitude, longitude,recog_num=5)
      
hospitals

# # 3. 결과 객체 생성
# emergency_result = md.EmergencyResult(
#     summary=summary,
#     danger_level=model_result["class_name"],
#     latitude=latitude,
#     longitude=longitude,
#     hospitals=hospitals
# )

# # 4. 결과를 딕셔너리로 변환
# result_dict = emergency_result.to_dict()

Whisper API Response :  다른 게 아니고요. 아이가 열이 나고 토하고 그래서요. 장염인가 싶어서요. 전에 소아과에서 받은 장염약을 먹였거든요. 그런데 아이가 점점 더 힘이 빠지네요. 물만 먹여도 토하고요. 탈수가 된 것 같아서요. 아무래도 응급실에 가봐야 할 것 같네요. 지금 가도 될까요?

summary response :  {
    "content": "아이가 열이 나고 토하며 점점 힘이 빠지고 있어서 응급실 방문이 필요할 것으로 보입니다.",
    "keyword": ["아이", "열", "응급실"]
}


[{'haversine_km': 1.111161722006186,
  'distance_km': 2.626,
  'hospital_name': '분당서울대학교병원',
  'address': '경기도 성남시 분당구 구미로173번길 82 (구미동, 분당서울대학교병원)',
  'emergency_type': '권역응급의료센터',
  'phone1': '031-787-2114',
  'phone3': '031-787-3119',
  'hospital_latitude': 37.35202597297615,
  'hospital_longitude': 127.12448354572744},
 {'haversine_km': 3.29996370427956,
  'distance_km': 4.707,
  'hospital_name': '대진의료재단분당제생병원',
  'address': '경기도 성남시 분당구 서현로180번길 20 (서현동)',
  'emergency_type': '지역응급의료센터',
  'phone1': '031-779-0114',
  'phone3': '031-779-0119',
  'hospital_latitude': 37.387871379956906,
  'hospital_longitude': 127.12132819639598},
 {'haversine_km': 4.7388990676761376,
  'distance_km': 5.973,
  'hospital_name': '국군수도병원',
  'address': '경기도 성남시 분당구 새마을로177번길 81 (율동)',
  'emergency_type': '지역응급의료기관',
  'phone1': '031-725-6075',
  'phone3': '031-725-6119',
  'hospital_latitude': 37.39186666666668,
  'hospital_longitude': 127.14858611111111},
 {'haversine_km': 5.794526496986809,
  'distan